In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import PIL.Image
import vgg1
# import download
import imagecomputations
# import lossfunction

In [2]:
#Function to get mean squared error between content image and mixed image; and styling image and mixed image
def mse(a, b):
    sqr = tf.square(a - b)
    redmean = tf.reduce_mean(sqr)
    return redmean

In [3]:
#loading content image
Cimage = imagecomputations.get_image('images/elon_musk.jpg')

In [4]:
#loading style image
Simage = imagecomputations.get_image('texture_outer/texture/texture_0011.jpg')

In [ ]:
styleids = list(range(13))
contentids = [4]

#declaring the layer number for which we want to use for the style image & content image 
def style_transfer(Cimage, Simage, contentids, styleids, weight_content, weight_style, num_iterations, step_size):
    
    #Init the model
    model = vgg1.VGG16()
    
    #creating the session ofr model and keeping it interactive to add nodes    
    session = tf.InteractiveSession(graph=model.graph)
    #Here initially the layer tensors for the content image are saved and the mse between the Content image and Mixed image is calculated in every layer
    feed_dict = model.create_feed_dict(image=Cimage)
    layers = model.get_layer_tensors(contentids)
    content_values = session.run(layers, feed_dict=feed_dict)
    with model.graph.as_default():
        layer_losses = []
#       filewriter = tf.summary.FileWriter('C:\\Users\\Raj Mehta\\Documents\\GitHub\\nst', loss.graph)
        for i in range(0, len(content_values)):
            loss = mse(content_values[i], layers[i])
            layer_losses.append(loss)
        total_loss_content = tf.reduce_mean(layer_losses)
    
    #Here first we calculate the gram matrix which is basically the correlation between different fiter responses and then we add this filter responses by taking the mse between them and Mixed image and gram image to mix the image with the style image
    #Here initially the layer tensors for the style image are saved and the mse between the Style image and Mixed image is calculated in every layer
    feed_dict = model.create_feed_dict(image=Simage)
    layers = model.get_layer_tensors(styleids)
    gram_layers = []
    with model.graph.as_default():
        for layer in layers:
            shape = layer.get_shape()
            num_channels = int(shape[3])
            matrix = tf.reshape(layer, shape=[-1, num_channels])
            gram = tf.matmul(tf.transpose(matrix), matrix)
            gram_layers.append(gram)
        values = session.run(gram_layers, feed_dict=feed_dict)
        layer_losses = []
        for i in range(0, len(values)):
            loss = mse(gram_layers[i], values[i])
            layer_losses.append(loss)
        total_loss_style = tf.reduce_mean(layer_losses)
    
    #Init of the learning rates
    adj_content = tf.Variable(1/1e-10, name='adj_content')
    adj_style = tf.Variable(1/1e-10, name='adj_style')

    session.run([adj_content.initializer,
                 adj_style.initializer])

    #calculating the learing rates ny taking the reciprocal of the respective losses
    update_adj_content = adj_content.assign(1.0 / (total_loss_content))
    update_adj_style = adj_style.assign(1.0 / (total_loss_style))

    #This is the combined loss calculated in particular iteration based on the weight of the content of the style image we want to be in mixed image
    loss_combined = weight_content * adj_content * total_loss_content + \
                    weight_style * adj_style * total_loss_style

    # this is the gradient calculated between the combined loss and the mixed image to use for back propoagation and update the image    
    gradient = tf.gradients(loss_combined, model.input)

    run_list = [gradient, update_adj_content, update_adj_style]

    Mimage = np.random.rand(*Cimage.shape) + 128

    #Iterations for constructing the mixed image
    for i in range(num_iterations):
        feed_dict = model.create_feed_dict(image=Mimage)
        
        #running the session and calculating the gradient
        grad, adj_content_val, adj_style_val =  \
        session.run(run_list, feed_dict=feed_dict)
        
#         filewriter = tf.summary.FileWriter('C:\\Users\\Raj Mehta\\Documents\\GitHub\\nst', grad.graph)

        grad = np.squeeze(grad)

        step_size_scaled = step_size / (np.std(grad) )

        #upadting the mixed image based on the gradient and the step size
        Mimage -= grad * step_size_scaled

        Mimage = np.clip(Mimage, 0.0, 255.0)

        if (i % 10 == 0):
            print()
            print("On Iteration {0:d} the output is: ".format(i))

            imagecomputations.plot_images(Cimage=Cimage,
                        Simage=Simage,
                        Mimage=Mimage)
            
    print()
    print("Final image:")
    imagecomputations.plot_final_image(Mimage)

    session.close()
    


In [ ]:
#runnning the model
style_transfer(Cimage=Cimage,
                     Simage=Simage,
                     contentids=contentids,
                     styleids=styleids,
                     weight_content=3.0,
                     weight_style=10.0,
                     num_iterations=30,
                     step_size=10.0)